<h1>Exploratory Data Analysis: Instructor Interview and Ratings Data</h1>

<p>In this exploratory data analysis, we'll be working with two datasets:</p>

<ul>
    <li>Instructor Interview Notes: A dataset consisting of first and second-round interview notes left on all instructors from last year, obtained from our Notion.</li>
    <li>Instructor Class Ratings: A dataset consisting of daily feedback ratings from students for all instructors, obtained from our database using a SQL query.</li>
</ul>

The purpose of this data analysis is to extract traits from the interview notes of top-performing instructors to aid our company in filtering out and finding candidates to prioritize. The underlying assumption of this analysis is that traits that past top-performing instructors had are a predictor of future top-performing instructors.

To start, let's import the necessary libraries and define some utility functions.

In [330]:
import pandas
import re

# Define utility functions
def is_valid_email(email):
    pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return re.match(pattern, email) is not None

def find_valid_email(a, b, c):
    if is_valid_email(a):
        return a
    if is_valid_email(b):
        return b
    if is_valid_email(c):
        return c
    return -1

The is_valid_email function checks if a given email is valid based on a regular expression pattern. The find_valid_email function returns the first valid email from three given input strings.

Now, let's read the datasets into Pandas DataFrames:

In [331]:
# Read datasets
interview_dataset = pandas.read_csv('/Users/blake/Downloads/2022_summer_instructor_interview_notes.csv')
ratings_dataset = pandas.read_csv('/Users/blake/Downloads/2022_summer_instructor_class_ratings.csv')

Next, we'll perform some data preparation on the ratings_dataset. We want to extract valid email addresses and keep only the 'overall_rating' and 'email' columns, since those will be the columns relevant to our analysis.

In [332]:
# Data preparation
ratings_dataset['email'] = ratings_dataset.apply(lambda row: find_valid_email(row['first_name'], row['last_name'], row['email']), axis=1)
ratings_dataset = ratings_dataset[['overall_rating', 'email']]

<h2>Data Processing: Merging and Filtering Instructor Data</h2>

In this section, we'll perform additional data processing on the ratings_dataset and interview_dataset before merging them. Then, we'll filter the merged dataset to focus on high-rated instructors.

First, let's group the ratings_dataset by email and calculate the mean rating for each instructor:

In [333]:
# Group ratings dataset by email and calculate the mean rating
ratings_dataset = ratings_dataset.groupby('email').mean().reset_index()

Next, we'll rename the 'E-Mail' column to 'email' in the interview_dataset and change the data type for the 'email' column:

In [334]:
# Rename columns and change the data type for email in the interview dataset
interview_dataset = interview_dataset.rename(columns={'E-Mail': 'email'})
interview_dataset['email'] = interview_dataset['email'].astype(str)

Now, we can merge the two datasets on the 'email' column:

In [335]:
# Merge the two datasets on 'email' column
merged_dataset = ratings_dataset.merge(interview_dataset, on='email', how='left')

After merging, we'll select the required columns ('overall_rating', '1st Round', and '2nd Round') and drop rows with missing data:

In [336]:
# Select required columns and drop rows with missing data
merged_dataset = merged_dataset[['overall_rating', '1st Round', '2nd Round']].dropna()

We'll sort the dataset by 'overall_rating' in descending order to make it easier to identify high-rated instructors:

In [337]:
# Sort the dataset by 'overall_rating' in descending order
merged_dataset = merged_dataset.sort_values('overall_rating', ascending = False)

Finally, we'll filter the dataset to focus on instructors with an overall rating greater than 4.5 (out of 5), which is the minimum cutoff for a high-performing instructor:

In [338]:
# Filter the dataset for instructors with an overall rating greater than 4.5
high_rated_instructors = merged_dataset[merged_dataset['overall_rating'] > 4.5].copy()

Let's display the top few high-rated instructors to get a sense of the data:

In [339]:
high_rated_instructors.head()

,overall_rating,1st Round,2nd Round
1,4.763485,Video Interview: https://www.youtube.com/watch...,"@April 5, 2022 12:45 PM (MDT) \nScore: 4\n\nMo..."
9,4.746032,Interviewer: @Cameron Jackson \nTime: @Februar...,Interviewer: @Alex Duffy \nDate/Time: @Februar...
7,4.737143,Interviewer: @Cameron Jackson \nTime: @March 2...,"@March 23, 2022 12:30pm PST\nscore: 3.5\n\n- h..."
19,4.714859,Video Interview: https://youtu.be/15G9ZRmouY4\...,"@April 8, 2022 12:02 PM (MDT) \nScore: 4 + I t..."
28,4.678161,"@March 17, 2022 11:30 AM (PDT) \nscore: 3\n\n-...","@March 25, 2022 3:15 PM (PDT) \nScore:3 \n\nMo..."


With the processed data in hand, we can proceed to analyze the high-rated instructors and draw insights from their interview notes and ratings.

<h2>Analyzing Instructor Traits: Identifying Prominent Strengths</h2>

In this section, we'll use the OpenAI API to analyze the interview notes of high-rated instructors and identify their most prominent strengths. This will give us insights into the qualities that contribute to their success. To do this, we can write a prompt to have the GPT-3 model output the trait that stands out the most about an applicant, given their interviewer's notes.

To start, let's import the required libraries and set up the OpenAI API key:

In [340]:
import os
import openai

# Set up OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')

Next, we'll define a function called get_prominent_trait that takes interview notes as input and returns the most prominent strength of the candidate:

In [341]:
# Define a function to get the prominent trait from the text
def get_prominent_trait(text: str) -> str:
    response = openai.Completion.create(
        model='text-davinci-003',
        prompt="""
        The following text consists of interview notes for a candidate for a summer internship. Based on the notes,
        what is the most prominent strength the candidate possesses. Output your answer
        as a single adjective with no punctuation at the beginning or end.
        
        <<NOTES>>
        {}
        
        Most prominent strength the candidate possesses. Must be an adjective with no punctuation at the beginning or end: 
        """.format(text),
        temperature=0.2,
        top_p=0.9
    )
    
    return response['choices'][0]['text'].strip()

Now, let's apply the get_prominent_trait function to the '1st Round' and '2nd Round' interview notes in the high_rated_instructors DataFrame:

In [342]:
# Add a new column 'trait' by applying the get_prominent_trait function to the interview notes
high_rated_instructors.loc[:, 'trait'] = high_rated_instructors.apply(lambda row: get_prominent_trait(row['1st Round'] + '\n' + row['2nd Round']), axis = 1)

Finally, let's display the first few records of the high_rated_instructors DataFrame with the added 'trait' column for inspection:

In [343]:
# Display the high-rated instructors
high_rated_instructors.head()

,overall_rating,1st Round,2nd Round,trait
1,4.763485,Video Interview: https://www.youtube.com/watch...,"@April 5, 2022 12:45 PM (MDT) \nScore: 4\n\nMo...",Motivation
9,4.746032,Interviewer: @Cameron Jackson \nTime: @Februar...,Interviewer: @Alex Duffy \nDate/Time: @Februar...,hardworking
7,4.737143,Interviewer: @Cameron Jackson \nTime: @March 2...,"@March 23, 2022 12:30pm PST\nscore: 3.5\n\n- h...",Nice
19,4.714859,Video Interview: https://youtu.be/15G9ZRmouY4\...,"@April 8, 2022 12:02 PM (MDT) \nScore: 4 + I t...",Motivated
28,4.678161,"@March 17, 2022 11:30 AM (PDT) \nscore: 3\n\n-...","@March 25, 2022 3:15 PM (PDT) \nScore:3 \n\nMo...",Motivated


By identifying the most prominent strengths of high-rated instructors, we can gain a better understanding of the qualities that contribute to their success and use this information to inform our selection and training processes.

In [344]:
high_rated_instructors['trait']

1        Motivation
9       hardworking
7              Nice
19        Motivated
28        Motivated
44     Motivational
13    Communicative
20        Motivated
47     Enthusiastic
14       Passionate
4         Motivated
42       Personable
17    Communicative
38    Communicative
32     Professional
6     Knowledgeable
25        Motivated
8         Motivated
24        Motivated
48         Engaging
Name: trait, dtype: object

<h2>Trait Analysis: Grouping and Visualizing Instructor Traits</h2>

In this section, we'll group the identified traits into categories based on their synonyms and visualize the distribution of these categories. This will help us understand the most common qualities among high-rated instructors.

First, let's define a function called get_trait_categories that takes a list of traits as input and returns a JSON dictionary with the categories and their frequencies:

In [345]:
# Define a function to get the trait categories from the list of traits
def get_trait_categories(traits: str) -> str:
    response = openai.Completion.create(
        model = 'text-davinci-003',
        prompt = """
        The following words are traits extracted from interview notes for a group of interns. Some words have similar meanings, such as "engaging" and "bright."" Please create 3-4 categories that group these words according to their synonyms, making sure to include every word. Name each category using the most representative word. Present the categories as a well-formatted JSON dictionary, with the category names as keys and the total frequency of words within each category as values.

        <<WORDS>>
        {}

        Create 3-4 categories as a properly-formatted JSON dictionary, ensuring all words are included, with category frequencies as values:

        """.format(traits),
        temperature = 0.2,
        top_p = 0.1,
        max_tokens = 256
    )
    return response['choices'][0]['text']


Next, we'll use the get_trait_categories function to group the traits from the high_rated_instructors DataFrame:

In [346]:
# Get the trait categories from the list of traits
traits_list = '\n'.join(high_rated_instructors['trait'].to_numpy())
trait_categories = get_trait_categories(traits_list)

Let's print the trait categories to inspect the results:

In [347]:
# Print the trait categories
print(trait_categories)

 {
            "Motivation": 8,
            "Communication": 3,
            "Professionalism": 1,
            "Knowledge": 1
        }


Now, we'll import the required libraries and load the trait categories JSON data:

In [348]:
import json
import plotly.express as px

# Load the trait categories JSON data
data = json.loads(trait_categories)

Finally, we'll visualize the distribution of trait categories using a pie chart:

In [349]:
# Visualize the trait categories using a pie chart
fig = px.pie(names = data.keys(), values = data.values(), title='Trait Categories Distribution')
fig.show()

<h2>Conclusion</h2>

Let's write a loop that outputs the traits of successful instructors, from most frequent to least frequent.

In [350]:
data = dict(sorted(data.items(), key=lambda item: item[1], reverse = True))

for i, j in data.items():
    print(f"{i}")

Motivation
Communication
Professionalism
Knowledge
